# Задача о рюкзаке  

##### проект №52  


### Выполнил: Михаил Цион, 795




# Основные определения  

> **Определение:** Машина распознаёт язык $A$ за время $T(n)$, если она принимает все слова, лежащие в $A$, отвергает все слова, не лежащие в $A$, и на каждом слове $x$ работает не больше $T (|x|)$ шагов.

> **Определение:** Классом ${\bf DTIME(T (n))}$ называется класс языков, которые распознаются за время $O(T(n))$ на детерминированной машине Тьюринга. Иными словами, время работы машины на любом слове длины $n$ не превосходит некоторой константы, умноженной на $T(n)$.

> **Определение:** Классом ${\bf P}$ называется множество языков, распознающихся за полиномиальное время. Формально он определяется так:
$$P =\bigcup\limits_{c=1}^{\infty} {\bf DTIME(n^c)} $$

> **Определение:** Классом ${\bf NTIME(T(n))}$ называется множество языков, распознаваемых на недетерминированной машине Тьюринга за время $O(T(n)$).

> **Определение:** Классом ${\bf NP}$ называется множество языков $A$, для которых существует функция $V(x, s)$ с булевыми значениями, вычислимая за полиномиальное время от длины первого аргумента, такая что:
*   Если $x \in A$, то $ \exists s \hspace{2pt} V(x, s) = 1$;
*   Если $x \notin A$, то $ \forall s \hspace{2pt} V (x, s) = 0$.


> **Определение:** Пусть $A$ и $B$ суть два языка. Тогда $A$ сводится по Карпу к $B$, если существует всюду определённая функция $f : \{0, 1\}^* \longrightarrow \{0, 1\}^*$ , вычислимая за полиномиальное время, такая что $x \in A \longleftrightarrow f (x) \in B$. Обозначение: $A \leq_p B$ . (Индекс $p$ означает полиномиальность).

> **Определение:** Язык $B$ является ${\bf NP}$**-трудным**, если для любого $A \in {\bf NP}$ выполнено $A \leq_p B$. Язык $B$ является ${\bf NP}$**-полным**, если он ${\bf NP}$**-трудный** и лежит в ${\bf NP}$.

# PROBLEM FORMULATION

Дано:
*   $n \in \mathbb{N}$ предметов
*   $w_{i} > 0 -$ вес $i-$го предмета
*   $c_{i} > 0 -$ cтоимость $i-$го предмета
*   $W \geq 0 -$ вместимость рюкзака  

Требуется найти такое множество предметов $M=\{(w_{j}, c_{j})| j \in \overline{1..n}\} \subseteq \bigcup\limits_{i=1}^{n} {\{(w_{i}, c_{i})\}}$, чтобы
$\sum\limits_{(w, c) \in M} w \leq W$, $\sum\limits_{(w, c) \in M} c \longrightarrow \max$. Допускается пустое множество предметов, т.е. $\forall{i} \in \overline{1..n} \space W < w_{i}$.

Далее, задачу о рюкзаке будем называть просто ${\bf KNAPSACK}$.

# KNAPSACK $\in$ NP-Complete

Прежде, чем доказывать $\textbf{NP}$**-полноту** для $\textbf{KNAPSACK}$, сделаем переформулировку задачи, сведя нашу задачу к задачи распознования языка: к условию добавим константу $P \geq 0$ и будем возвращать **1**, если $\exists \hspace{2pt} I \subset \overline{1..n}$ и выполнено:
*   $\sum\limits_{i \in I} w_i \leq W$
*   $\sum\limits_{i \in I} c_i \geq P$,  
иначе возращаем **0**.

## KNAPSACK $\in$ NP 

Для доказательства принадлежности к $\textbf{NP}$ воспользуемся сертификатным определением.  
Возмем сертификат $s-$ поднабор $\overline{1..n}$. Если $\exists{I}$, удовлетворяющий требованиям выше, то $\exists{s}$, который подадим на вход, как второй аргумент $V(\cdot; \cdot)$, а $V(\cdot; s)$ проверит, что $I$ соответствует соотношениям выше.  
   
**Следовательно, KNAPSACK принадлежит NP.**

## KNAPSACK $\in$ NP-Hard

Для начала, рассмотрим частный случай задачи о рюкзаке $-$ $\textbf{SUBSETSUM}$.
Пусть дано множество $S$, состоящее из $n$ чисел. Нужно выяснить, существует ли его подмножество с суммой чисел, равной $s$. Это можно рассматривать как $\textbf{KNAPSACK}$, где $\forall{i}: c_{i} = w_{i}, P = W = s$. Будем обозначать данную задачу как $(S, s)$.

Сведем $\textbf{NP}$-трудную задачу о выполнимости булевой формулы в форме 3-КНФ ($\textbf{3-SAT}$) к $\textbf{SUBSETSUM}$, а т.е. и к $\textbf{KNAPSACK}$.

Пусть задана 3-КНФ формула $\phi$ от $n$ переменных, состоящая из $k$ пар скобок $C_i$. Считаем, что ни одна пара скобок не содержит одновременно переменную и ее отрицание, и каждая переменная входит хотя бы в одну пару скобок.

Построим сводящую функцию $f: \phi \Rightarrow (S, s) $: \\
Для каждой переменной $x_i$ и для каждой пары скобок $C_j$ создадим по два числа в десятичной системе счисления, каждое длиной $n+k$ цифр. Эти числа образуют $S$. Присвоим каждому разряду полученных чисел метку (одинаковую для всех чисел), соответствующую либо переменной, либо паре скобок. Метки, соответствующие парам скобок, присвоены $k$ младшим разрядам чисел.
*  В числе $s$ все разряды, соответствующие переменным, установим 1, а оставшиеся сделаем равными 4.
*  Каждой переменной $x_i$ соответствуют два числа из $S$: $v_i$ и $u_i$. В обоих числах установим разряд, соответствующий данной переменной равным $1$, все остальные разряды переменных -- $0$. Для числа $v_i$ установим разряды, соответствующие скобках, в которые входит $x_i$, равными $1$, все остальные -- $0$. В $u_i$ же сделаем то же самое, но с $\lnot x_i$
*  Каждой паре скобок $C_{j}$ соответствуют два числа из $S$ : $d_{i}$ и $e_{i}$. Оба этих числа содержат 0 во всех разрядах, кроме соответствующего $C_{j}$. В этом разряде у $d_{i}$ 1, а у $e_{i}$ 2.  

Докажем корректность определенной выше функции, т.е. то, что мы действительно сводим $\textbf{3-SAT}$ к $\textbf{SUBSETSUM}$, и делаем это полиномиально.
*  Полученное множество $S$ состоит из $2(n + k)$ чисел, выставить каждый разряд точно возможно за полином, значит и само сведение выполняется за полином.
*  Пусть в $S$ существует подмножество $S'$, т.ч. $\sum\limits_{s_{i} \in S'} s_{i} = s$. Докажем, что $\phi$ выполнима. Заметим, набору $S'$ обязательно принадлежит ровно одно из $v_{i}$ и $u_{i}$, т.к. в $s$ соответствующий $x_{i}$ разряд равен $1$. Если принадлежит $u_{i}$ сделаем $x_{i} = 1$, иначе $-$ $0$. Далее, т.к. разряды, соответствующие скобкам равны $4$, а с помощью $d_{j}$ и $e_{j}$ мы можем получить максимум число $3$ в разряде, то в любой скобке есть хотя бы один выбранный нами терм ($x_{i}$, если в $S'$ лежит $u_{i}$, или его отрицание в другом случае). А значит, каждый дизъюнкт верен. Значит, верна и исходная формула.
*  Пусть $\phi$ выполнима, т.е. существует набор значений переменных $y_{1},...,y_{n}$, при которых формула истинна. Пусть $S$ и $s$ получены после сводимости. Нужно доказать, что в $S$ существует подмножество (обозначим за $S'$) с суммой, равной $s$. Для каждой переменной $x_{i}$ добавим $v_{i}$ в $S'$, если $y_{i} = 1$, иначе $-$ $u_{i}$. Заметим, что для каждого разряда, соответствующего скобках, в уже набранной части $S'$ есть не менее одного и не более трех чисел, у которых в данном разряде стоит $1$. Значит для каждого соответствующего паре скобок $C_{j}$ разряда мы сможем выбрать одно или оба числа $d_{j}$ и $e_{j}$ так, чтобы сумма в данном разряде стала равной $4$. Добавим их в $S'$. Заметим, что суммы во всех разрядах, соответствующих переменным, равны $1$, так как для каждого $i$ выбиралось только одно число из $v_{i}$ и $u_{i}$. Значит, теперь $\sum\limits_{s_{i} \in S'} s_{i} = s$.  

Значит, доказали **NP**-трудность задачи $\textbf{SUBSETSUM}$, а, значит, и **NP**-трудность задачи о рюкзаке. **Следовательно, KNAPSACK принадлежит к NP-трудным.**

# Bruteforce algorithm

*Тривиальным* решением задачи о рюкзаке есть полный перебор всех подмножеств множества предметов и отыскания того набора, вес которого не превосходит установленного порога, а суммарная стоимость максимальна.  

**Время работы алгоритма:** $O(n \cdot 2^{n})$, т.к. всего $2^{n}$ подмножеств, $n-$элементного множества, проверка каждого подмножества занимает не более $n$ шагов.

# Implementation

In [0]:
import functools
from itertools import (
  chain,
  combinations)
import math
import operator

In [0]:
class Item(object):

  def __init__(self, weight=0, cost=0, *args, **kwargs):
    self.weight = weight
    self.cost = cost

  def __add__(self, other):
    item = Item(weight=self.weight+other.weight, cost=self.cost+other.cost)
    if 'rounded_cost' in self.__dict__.keys():
      item.rounded_cost = self.rounded_cost + other.rounded_cost
    return item
 
  def __repr__(self):
    item_repr = "Item("
    for (k, v) in self.__dict__.items():
      item_repr += "{0}: {1}; ".format(k, v)
    item_repr = item_repr[:-2] + ")"
    return item_repr

Лямбда-функция $\verb|all_indexes_subsets|$ от $\verb|n|$ возращает список элементов из $2^{\{0...n-1\}}-$ множество множеств всех возможных индексов предметов (в теоретическом изложении индексация начинается с $1$, а в реализации c $0$).

In [0]:
all_indexes_subsets = lambda size: filter(
    lambda subset: len(subset) > 0,
    chain(*[combinations(range(size), i) for i in range(size+1)])
)

In [0]:
def launch_bruteforce(items, max_weight) -> tuple:
  total_cost, total_weight = 0, 0
  for index_subset in all_indexes_subsets(len(items)):
    sub_total_cost, sub_total_weight = 0, 0
    for index in index_subset:
      sub_total_cost += items[index].cost
      sub_total_weight += items[index].weight
    if sub_total_cost > total_cost and sub_total_weight <= max_weight:
      total_cost, total_weight = sub_total_cost, sub_total_weight
  return total_weight, total_cost

# Greedy algorithm

**Алгоритм:**  
*  Удалим все объекты, вес которых больше, чем $W$. Если для оставшихся $n^{'}\leq n$ объектов выполнено: $\sum\limits_{i=1}^{n^{'}} w_{i} \leq W$, то все эти объекты и являются ответом жадного алгоритма. Иначе переходим к следующим пунктам.
*  Обозначим $p_{i} := \frac{c_{i}}{w_{i}}$ и отсортируем элементы ${x_i}$ по убыванию $p_i$
*  Выберем $k:\space\sum\limits_{i=1}^{k} w_i \leq W < \sum\limits_{i=1}^{k + 1} w_i$.
*  Первые $k$ объектов после сортировки являются ответом жадного алгоритма..  

**Заметим**, что для ответа такого алгоритма выполнено (обозначим за $C'$ значение истинного ответа на задачу):
$$
\sum\limits_{i=1}^{k} c_{i} \leq C' < \sum\limits_{i=1}^{k+1} c_{i}
$$  

**Доказательство:**  
Обозначим $W' = \sum\limits_{i=1}^{k + 1} w_i$.  
Для задачи с максимальным весом $W'$ решением будет будут первые $(k+1)$ объект после сортировки, с общей стоимостью $\sum\limits_{i=1}^{k + 1} c_i$, т.к. эти предметы обладают большей ценой за единицу измерения веса и занимают весь вес. Значит, исходная задача не могла иметь решение с большей стоимостью, т.к. $W \leq W'$.  

**Время работы алгоритма:**  
$O(n \log n)$ или $O(n^{2})$, в зависимости от используемой сортировки.

# Implementation

In [0]:
class GreedySolution(object):
    
    def __init__(self, size=0, weight=0, cost=0, *args, **kwargs):
        self.size = size
        self.weight = weight
        self.cost = cost

    def __iadd__(self, item: Item):
      self.size += 1
      self.weight = item.weight
      self.cost = item.cost
      return self

    def __repr__(self):
      item_repr = "GreedySolution("
      for (k, v) in self.__dict__.items():
        item_repr += "{0}: {1}; ".format(k, v)
      item_repr = item_repr[:-2] + ")"
      return item_repr

In [0]:
def launch_greedy(items, max_weight) -> GreedySolution:
  lifting_items = list(filter(lambda item: item.weight <= max_weight, items))
  solution = GreedySolution()
  if len(lifting_items) == 0:
    return solution
  solution += functools.reduce(operator.add, lifting_items)
  if solution.weight > max_weight:
    lifting_items.sort(key=lambda item: -item.cost/item.weight)
    solution = GreedySolution()
    while solution.weight <= max_weight and solution.size < len(lifting_items):
      if solution.weight + lifting_items[solution.size].weight <= max_weight:
        solution += lifting_items[solution.size]
      else:
        break
  return solution

# Pseudo-polynomial algorithm

В полиномиальной схеме приближения задачи о рюкзаке будет использоваться псевдополиномиальный алгоритм c целыми стоимостями.
> **Определение:** Алгоритм называется псевдополиномиальным, если работает полиномиально от размера входных данных, записанных в унарном виде.

Обозначим за $w(k, c)$ наименьшей вес подмножества первых $k$ объектов, имеющего стоимость $c$. Имеют место следующие соотношения для нашего алгоритма:
*  $w(0, 0) = 0$
*  $w(0, c) = +\infty$
*  $w(k+1, c) = \min (w(k, c), \hspace{2pt} w(k, c - c_{k+1}) + w_{k + 1})$

Нетрудно заметить, что посторенный алгоритм относится к алгоритмам из двумерной динамики.

**Используемая память:**  
Пусть мы обладаем знанием, что максимальная стоимость $C' \leq T$, тогда всего потребуется $(n+1)T$ состояний, т.е. $O(T)$ памяти.  

**Время работы алгоритма:**  
Работает же алгоритм за $O(n \cdot T)$, т.к. при добавлении каждого объекта достаточно просмотреть $T$ состояний. Понятно, что $T$ не больше $\sum_{i=1}^{n}c_{i} \leq n\cdot \max\limits_{1\leq i\leq n} c_{i}$. Следовательно, итоговое время работы полиномиально.

# Implementation

In [0]:
class DynamicState(object):
  
  __slots__ = ['min_weight', 'cost', 'last_item']

  def __init__(self, min_weight=math.inf, cost=0, last_item=-1, 
               *args, **kwargs):
    self.min_weight = min_weight
    self.cost = cost
    self.last_item = last_item

In [0]:
def launch_dynamic(items, max_weight, max_cost=0) -> list:
  if max_cost == 0:
    max_cost = functools.reduce(
        function=lambda total, item: total + item.cost,
        iterable=items)
  states = [DynamicState() for _ in range(max_cost+1)]
  states[0].min_weight = 0
  for (index, item) in enumerate(items):
    for cost in range(max_cost, item.cost, -1):
      if item.weight + states[cost-item.cost].min_weight < \
      states[cost].min_weight:
        states[cost] = DynamicState(
            item.weight + states[cost-item.cost].min_weight,
            item.cost + states[cost-item.cost].cost, index
        )
  return states

# Polynomial approximation

> **Определение:** Пусть у нас есть $\textbf{NP}$-сложная задача максимизации функции $f$, $E-$ ее входные данные, $C_{E}-$ оптимальный ответ на входе $E$. Алгоритм называется **$(1 - \varepsilon)-$приближением** для решения данной задачи, если на входе $(E, \varepsilon)$ он выдает такое ответ $C_{E}^{'}$, что $C_{E}^{'} \geq (1 - \varepsilon) \cdot C_{E}$. Если алгоритм работает за полиномиальное время от размера входа и $\frac{1}{\varepsilon}$, то он называется **полиномиальным приближением**.  

В предположении $\textbf{P} \neq \textbf{NP}$ задача о рюкзаке не имеет полиномиального алгоритма решения. Но при этом, если придумать алгоритм полиномиального приближения, то за полиномиальное время мы научимся приближенно решать задачу, получая ответ заданной точности.  

Обозначим максимально возможную стоимость предметов, общего веса $\leq W$ за $C'$. Рассмотрим два алгоритма:  


*   основанный на подходе динамического программирования
*   алгоритм Ибарры-Кима



## Polynomial approximation by dynamic programming

**Алгоритм:**
*  Удалим все предметы, масса которых больше максимально возможной.
*  Определим $c_{0} := \max\limits_{1\leq i\leq n} c_{i}$ (под $n$ подразумевается количество уже отфильтрованных предметов). 
*  Заменим стоимость каждого предмета $c_{i}$ на $c_{i}^{'} := \left[\frac{c_{i}}{\delta}\right]$, где нормирующий коэффициент $\delta$ равен $\frac{\varepsilon \cdot c_{0}}{n}$, а $n-$ это как и всегда количество предметов. 
*  Для новых стоимостей найдем наилучшее решение псевдополиномиальным алгоритмом, предметы из этого решения и будут ответом (обозначим их индексы за $I$).  

> **Теорема.** Построенный алгоритм является полиномиальным приближения задачи о рюкзаке. Временная сложность алгоритма $O(\frac{n^{3}}{\varepsilon})$, пространственная сложность $O(\frac{n^{2}}{\varepsilon})$.  

> **Доказательство.** Докажем, что алгоритм действительно строит $(1 - \varepsilon)-$приближение. Мы знаем, что псевдополиномиальное приближение находит точное решение. Т.е. переходя от начальных нецелых стоимостей к целым, путем взятия целой части при делении на $\delta$, мы найдем точное решение в стоимостях $c^{'}$. Пусть $I_{0}-$индексы предметов из истинного решения исходной задачи, $I-$индексы решения задачи в стоимостях $c^{'}>0$, тогда верно:
$$
\sum\limits_{i \in I} \frac{c_{i}}{\delta} \geq \sum\limits_{i \in I} \left[\frac{c_{i}}{\delta}\right] \geq \sum\limits_{i \in I} c_{i}^{'} \geq \sum\limits_{j \in I_{0}} c_{j}^{'}
.$$
Значит,
$$
\sum\limits_{i \in I} c_{i} \geq \delta \cdot \sum\limits_{j \in I_{0}} c_{j}^{'}
.$$
И из неравенства:
$$
\sum\limits_{i \in I_{0}} (c_{i} - \delta \cdot c_{i}^{'}) = \sum\limits_{i \in I_{0}} (c_{i} - \delta \cdot \left[\frac{c_{i}}{\delta}\right]) = \delta\cdot\sum\limits_{i \in I_{0}} (\frac{c_{i}}{\delta} - \left[\frac{c_{i}}{\delta}\right]) \leq \delta\cdot\sum\limits_{i \in I_{0}} 1 = \delta \cdot |I_{0}| \leq \delta \cdot n = \frac{\varepsilon \cdot c_{0}}{n} \cdot n \leq \varepsilon \cdot c_{0} \leq \varepsilon \cdot C^{'}
.$$
Получаем оценку:
$$
\sum\limits_{i \in I} c_{i}\geq \delta \cdot \sum\limits_{j \in I_{0}} c_{j}^{'} \geq \sum\limits_{i \in I_{0}} c_{i} - \varepsilon \cdot C^{'} = (\frac{\sum\limits_{i \in I_{0}} c_{i}}{C^{'}} - \varepsilon) \cdot C^{'} \geq (1 - \varepsilon) \cdot C^{'}
,$$
т.к. $\sum\limits_{i \in I_{0}} c_{i} \geq C^{'}.$  
А, значит, найденный нами набор является $(1-\varepsilon)-$приближением ответа. Заметим, т.к. $\frac{c_{0}}{\delta} = \frac{n}{\varepsilon}$, то максимальная стоимость решения не больше, чем $\frac{n^{2}}{\varepsilon}$, следовательно, и алгоритм работает за $O(\frac{n^{3}}{\varepsilon})$. При этом пространственная сложность $O(\frac{n^{2}}{\varepsilon})$


## Implementation

Функция $\verb|compute_delta_normaized_items|$ высчитывает отнормированную по $\delta$ стоимость предмета.

In [0]:
def compute_delta_normaized_items(items, delta) -> list:
  for item in items:
    item.rounded_cost = int(item.cost / delta)
  return items

Вспомогательная функция $\verb|compute_dynamic_approx|$ вычисляет 

In [0]:
def compute_dynamic_approx(items, delta, max_cost=0) -> list:
  if max_cost == 0:
    max_cost = sum((item.rounded_cost for item in items))
  states = [DynamicState() for _ in range(max_cost+1)]
  states[0].min_weight = 0
  for (index, item) in enumerate(items):
    for cost in range(max_cost, item.rounded_cost-1, -1):
      if item.weight + states[cost - item.rounded_cost].min_weight < \
      states[cost].min_weight:
        states[cost] = DynamicState(
          item.weight + states[cost - item.rounded_cost].min_weight,
          item.cost + states[cost - item.rounded_cost].cost, index)
  return states

In [0]:
def launch_polynomial_approx_by_dynamic(items, max_weight, precision):
  filtered_items = list(filter(lambda item: item.weight <= max_weight, items))
  c0 = max(filtered_items, key=operator.attrgetter('cost')).cost
  delta = c0 * precision / len(filtered_items)
  if 'rounded_cost' not in filtered_items[0].__dict__.keys():
    compute_delta_normaized_items(filtered_items, delta)
  states = compute_dynamic_approx(filtered_items, delta, 
                                  max_cost=int(len(filtered_items)*c0/delta))
  max_cost, rounded_cost = 0, 0
  for (index, state) in enumerate(states):
    if state.min_weight > max_weight:
      continue
    if state.cost > max_cost:
      max_cost = state.cost
      rounded_cost = index
  result = []
  while rounded_cost > 0:
    result.append(filtered_items[states[rounded_cost].last_item])
    rounded_cost -= result[-1].rounded_cost
  return result

## Polynomial approximation by Ibarra-Kim

Алгоритм гарантированно получает $(1-\varepsilon)-$приближение в случае, когда $C' \geq \max{\{\sum\limits_{i=1}^{k} c_{i}; c_{k+1}\}}$, где предметы отсортированны как в жадном алгоритме, $k$ взято из него же. Для этого случая будет доказана его корректность.  
Суть алгоритма состоит в разделении предметов на **дорогие** и **дешевые**. Для дорогих предметов воспользуемся псевдополиномиальным алгоритмом. Для дешевых, учитывая оставшийся вес от выбранных дорогих, воспользуемся жадным алгоритмом.  

**Алгоритм**:
*  Обозначим ${\bf C_{m}} = \frac{1}{2} \cdot \sum\limits_{i=1}^{k+1} c_{i}$, где $k$ взято из жадного алгоритма (перед этим отсортировали предметы по $p_{i}=\frac{c_{i}}{w_{i}})$. Имеем ${\bf C_{m}} \leq C' \leq 2 \cdot {\bf C_{m}}$, при условии, что $C' \geq \max{\{\sum\limits_{i=1}^{k} c_i; c_{k + 1}\}}$.
*  Разделим предметы **дорогие** и **дешевые**. Если $c_i \geq \varepsilon \cdot {\bf C_m} $, то он дорогой, иначе $-$ дешевый. Пусть дорогие объекты имеют индексы: $1,...,l$, дешевые $-$ $l+1,...,n$.
*  Обозначим $\delta := \varepsilon^2 \cdot {\bf C_{m}}$. Заменим стоимость каждого дорогого на целое число: $c_{i} \rightarrow \left[\frac{c_i}{\delta}\right]$
*  Обозначим $C_{lim} := [\frac{2 \cdot {\bf C_{m}}}{\delta}]$, где $m-$ количество дорогих элементов. 
*  Далее $\forall c \in \{0,...,C_{lim}\}$: для дорогих $-$ псевдополиномиальным алгоритмом находим $w(m, c)$ (мы нашли минимальный вес дорогих имеющих стоимость не меньше $c$), обозначим соответствующую им стоимость за $C_{c}^{1}$; дешевых $-$ жадным алгоритмом находим объекты, дающие максимальную стоимость, имеющих вес не больше $(W - w(m, c))$, обозначим соответствующую им стоимость за $C_{c}^{2}$; суммируем найденные стоимости.
*  Нашим ответом будет $\max\limits_{0\leq j\leq C_{lim}} \{C_{j}^{1} + C_{j}^{2}\}$.   

> **Теорема.** При условии $C' \geq \max{\{\sum\limits_{i=1}^{k} c_{i}; c_{k + 1}\}}$ данный алгоритм строит $(1-\varepsilon)-$приближение истинного решения. Временная сложность алгоритма $O(n \cdot \log n + \frac{n}{\varepsilon^2})$.  

> **Доказательство.**  Докажем, что алгоритм строит $(1-\varepsilon)-$приближение.  
Для **дорогих** предметов верно неравенство, доказанное ранне (в силу того, что целая часть от частного, умноженная на делитель, отличается от делимого менее, чем на делитель):
$$
c_{i} - \delta \cdot c_{i}^{'} \leq c_{i} - \delta \cdot \left[\frac{c_{i}}{\delta}\right] < \delta
$$
При этом, $\delta = \varepsilon^2 \cdot {\bf C_{m}}$, и для любого дорогого предмета выполнено: $c_{i} \geq \varepsilon \cdot {\bf C_{m}}$. Из этого получаем, что:
$$
c_{i} - \delta c_{i}^{'} < \varepsilon^2 \cdot {\bf C_{m}} \leq \varepsilon^2 \cdot \frac{c_{i}}{\varepsilon} = \varepsilon \cdot c_{i}
$$
То есть суммарная ошибка для дорогих не больше, чем $2 \cdot \varepsilon \cdot {\bf C_{m}}$.  
Для **дешевых** предметов мы применяем жадный алгоритм, значит, верно, что найденное решение отличается от истинного не более, чем, на $c_{l+1}$, где $l-$ размер множества предметов, полученного жадным алгоритмом для дешевых предметов, т.е. не более, чем на $\varepsilon \cdot {\bf C_{m}}$. Значит, суммарная ошибка на всех предметах не более $3 \cdot \varepsilon \cdot {\bf C_{m}} \leq 3 \cdot \varepsilon \cdot C^{'}$.  
Осталось доказать, что алгоритм работает за  $O(n \cdot \log n + \frac{n}{\varepsilon^2})$. Сложность сортировки, поиска ${\bf C_{m}}$ и разделения предметов составляет $O(n \cdot \log n)$. Сложность подсчёта $w(n, c)$ составляет $O({\bf C_m} \cdot \frac{n}{\delta}) = O(\frac{n}{\varepsilon^{2}})$. Тогда общая **сложность алгоритма**: $O(n \cdot \log n + \frac{n}{\varepsilon^2})$.

## Implementation

In [0]:
def launch_ibarra_kim(items, max_weight, precision) -> list:
  precision /= 3  # см. "суммарная ошибка" в доказательстве выше
  filtered_items = list(filter(lambda item: item.weight <= max_weight, items))
  filtered_items = sorted(filtered_items, 
                          key=lambda item: -item.cost/item.weight)
  solution = launch_greedy(filtered_items, max_weight)
  if solution.size == len(filtered_items):
    return filtered_items
  C_m = 0.5 * (solution.cost + filtered_items[solution.size].cost)  
  expensive_threshold = lambda item: item.cost >= precision*C_m
  expensive_items = list(filter(expensive_threshold, filtered_items))
  cheap_threshold = lambda item: not expensive_threshold(item)
  cheap_items = list(filter(cheap_threshold, filtered_items))
  compute_delta_normaized_items(filtered_items, C_m * precision**2)
  if len(expensive_items) == 0:
    return launch_greedy(expensive_items, max_weight)
  dynamic_states = compute_dynamic_approx(expensive_items, C_m * precision**2, 
                                          int(2/precision**2+1))
  max_cost = 0
  expensive_max_rounded_cost = 0
  greedy_solution_size = 0
  for rounded_cost in range(len(dynamic_states)):
    if dynamic_states[rounded_cost].min_weight > max_weight:
      continue
    local_greedy = launch_greedy(
      items=cheap_items,
      max_weight=max_weight-dynamic_states[rounded_cost].min_weight
    )
    if local_greedy.cost + dynamic_states[rounded_cost].cost > max_cost:
      max_cost = local_greedy.cost + dynamic_states[rounded_cost].cost
      expensive_max_rounded_cost = rounded_cost
      greedy_solution_size = local_greedy.size
  current_cost = expensive_max_rounded_cost
  maximum_set_items = []
  while current_cost > 0:
    maximum_set_items.append(
      expensive_items[dynamic_states[current_cost].last_item])
    current_cost -= maximum_set_items[-1].rounded_cost
  maximum_set_items.reverse()
  for i in range (greedy_solution_size):
    maximum_set_items.append(cheap_items[i])
  return maximum_set_items

# Testing  



In [0]:
import random


class GeneratorsFactory:

  @staticmethod
  def get_power_of_two_items_set(size, start_weight=1):
    def items_generator():
      nonlocal size, start_weight
      for _ in range(size):
        yield Item(start_weight, start_weight)
        start_weight *= 2
    return items_generator

  @staticmethod
  def get_sequential_items_set(size, start_weight=1):
    def items_generator():
      nonlocal size, start_weight
      for value in range(start_weight, size+start_weight):
        yield Item(value, value)
    return items_generator

  @staticmethod
  def get_random_items_set(size, lrange=1, rrange=100):
    def items_generator():
      nonlocal size, lrange, rrange
      for _ in range(size):
        yield Item(weight=random.randint(lrange, rrange), 
                  cost=random.randint(lrange, rrange))
    return items_generator

In [0]:
import time


class TestTimer:
 
  benchmark_result = None

  def __init__(self, label, test_num):
    self.label = label
    self.test_num = test_num

  def __enter__(self):
    self.start = time.time()
    return self

  def __exit__(self, exc_type, exc_val, exc_tb):
    end = time.time()
    runtime = end - self.start
    TestTimer.benchmark_result = runtime

  @staticmethod
  def drop_benchmark_data():
    TestTimer.benchmark_result = None

In [0]:
import copy


class TestRunner:

  def __init__(self):
    self.test_num = 0
    self._tests_results = {}

  def _store(self, algo, label, exectime, result, max_weight, precision):
    if self._tests_results.get(self.test_num, None) is None: 
      self._tests_results[self.test_num] = {
          "label": label,
          "max_weight": max_weight,
          "precision": precision,
          "execution": {}
      }
    self._tests_results[self.test_num]["execution"][algo] = {}
    self._tests_results[self.test_num]["execution"][algo]["time"] = exectime
    self._tests_results[self.test_num]["execution"][algo]["result"] = result

  def _reduce(self, items):
    return {"weight": sum((item.weight for item in items)), 
            "cost": sum((item.cost for item in items))}

  def run(self, label, generator, allow_bruteforce, max_weight, precision):
    self.test_num += 1
    items = list(generator())
    if allow_bruteforce:
      bruteforce_result = None
      with TestTimer(label, self.test_num):
        bruteforce_result = launch_bruteforce(items, max_weight)
      self._store("bruteforce", label, exectime=TestTimer.benchmark_result,
                  result={"weight": bruteforce_result[0], 
                          "cost": bruteforce_result[1]},
                  max_weight=max_weight, 
                  precision=precision)
    poly_approx_result = None
    with TestTimer(label, self.test_num):
      poly_approx_result = launch_polynomial_approx_by_dynamic(items, 
                                                               max_weight,
                                                               precision)
    self._store("dynamic", label, exectime=TestTimer.benchmark_result,
                result=self._reduce(poly_approx_result), max_weight=max_weight, 
                precision=precision)
    ibarra_kim_result = None
    with TestTimer(label, self.test_num):
      ibarra_kim_result = launch_ibarra_kim(items, max_weight, precision)
    self._store("ibarra-kim", label, exectime=TestTimer.benchmark_result,
                result=self._reduce(ibarra_kim_result), max_weight=max_weight, 
                precision=precision)

  @property
  def tests_results(self):
    return copy.deepcopy(self._tests_results)

In [0]:
runner = TestRunner()

Проведем несколько серий тестовых запусков. Поделим серии на $3$ типа:


1.   $c_{i} = p_{i}-$генерируются последовательно на $\{a;...;a+n\}$
2.   $c_{i} = p_{i}-$генерируются степенями двойки, т.е. на $\{x|x=2^{b}, a\leq b\leq a+n\}$
3.   $c_{i},p_{i}-$генерируются рандомом

### Запустим серии первого типа.

In [0]:
runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(10),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.5
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(10),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.1
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(10),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.05
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(10),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.01
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(10),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.001
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(100),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.5
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(100),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.1
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(100),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.05
)

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(100),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.01
) 

runner.run(
    label="sequential",
    generator=GeneratorsFactory.get_sequential_items_set(100),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.001
) 

### Запустим серии второго типа.

In [0]:
runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(10),
    allow_bruteforce=True,
    max_weight=777,
    precision=0.5
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(10),
    allow_bruteforce=True,
    max_weight=777,
    precision=0.1
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(10),
    allow_bruteforce=True,
    max_weight=777,
    precision=0.05
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(10),
    allow_bruteforce=True,
    max_weight=777,
    precision=0.01
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(10),
    allow_bruteforce=True,
    max_weight=777,
    precision=0.001
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(100),
    allow_bruteforce=False,
    max_weight=2**69,
    precision=0.5
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(100),
    allow_bruteforce=False,
    max_weight=2**69,
    precision=0.1
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(100),
    allow_bruteforce=False,
    max_weight=2**69,
    precision=0.05
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(100),
    allow_bruteforce=False,
    max_weight=2**69,
    precision=0.01
)

runner.run(
    label="power of two",
    generator=GeneratorsFactory.get_power_of_two_items_set(100),
    allow_bruteforce=False,
    max_weight=2**69,
    precision=0.001
)

### Запустим серии третьего типа.

In [0]:
runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(10, 1, 40),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.5
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(10, 1, 40),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.1
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(10, 1, 40),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.05
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(10, 1, 40),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.01
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(10, 1, 40),
    allow_bruteforce=True,
    max_weight=40,
    precision=0.001
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(100, 1, 400),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.5
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(100, 1, 400),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.1
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(100, 1, 400),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.05
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(100, 1, 400),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.01
)

runner.run(
    label="random",
    generator=GeneratorsFactory.get_random_items_set(100, 1, 400),
    allow_bruteforce=False,
    max_weight=400,
    precision=0.001
)

### Проверим результат тестирования.

In [20]:
import pandas


columns=[
  "Данные", "Пороговый вес", "Точность", "Перебор: вес", "Перебор: стоимость",
  "Перебор: время", "Динамика: вес", "Динамика: стоимость", "Динамика: время", 
  "Ибарра-Ким: вес", "Ибарра-Ким: стоимость", "Ибарра-Ким: время"
]
df = pandas.DataFrame(columns=columns)

for test in runner.tests_results.values():
  df = df.append(dict(zip(
    columns, [
      test["label"],
      test["max_weight"],
      test["precision"],
      test["execution"].get("bruteforce", {"result": {"weight": "-"}})["result"]["weight"],
      test["execution"].get("bruteforce", {"result": {"cost": "-"}})["result"]["cost"],
      test["execution"].get("bruteforce", {"time": "-"})["time"],
      test["execution"]["dynamic"]["result"]["weight"],
      test["execution"]["dynamic"]["result"]["cost"],
      test["execution"]["dynamic"]["time"],
      test["execution"]["ibarra-kim"]["result"]["weight"],
      test["execution"]["ibarra-kim"]["result"]["cost"],
      test["execution"]["ibarra-kim"]["time"],
    ]
  )), ignore_index=True)

df

,Данные,Пороговый вес,Точность,Перебор: вес,Перебор: стоимость,Перебор: время,Динамика: вес,Динамика: стоимость,Динамика: время,Ибарра-Ким: вес,Ибарра-Ким: стоимость,Ибарра-Ким: время
0,sequential,40,0.500,40,40,0.00268793,40,40,0.001374,55,55,0.000056
1,sequential,40,0.100,40,40,0.000977278,40,40,0.002582,55,55,0.000049
2,sequential,40,0.050,40,40,0.00103474,40,40,0.006227,55,55,0.000088
3,sequential,40,0.010,40,40,0.000971317,40,40,0.026681,55,55,0.000103
4,sequential,40,0.001,40,40,0.0011332,40,40,0.305207,55,55,0.000068
5,sequential,400,0.500,-,-,-,400,400,0.397277,5050,5050,0.000362
6,sequential,400,0.100,-,-,-,400,400,2.074235,5050,5050,0.000296
7,sequential,400,0.050,-,-,-,400,400,4.100209,5050,5050,0.000295
8,sequential,400,0.010,-,-,-,400,400,20.578099,5050,5050,0.000290
9,sequential,400,0.001,-,-,-,400,400,207.516027,5050,5050,0.000370


# Conclusion

*   Задача о рюкзаке является **NP-полной**.
*   Алгоритмы полиномиального приближения дают хорошую аппроксимацию ответа.

# References

1.  Д.В. Мусатов, черновики лекций по дисциплине Сложности вычислений.

2.  М.Гэри, Д.Джонсон. *Вычислительные машины и труднорешаемые задачи*. М: Мир,
1982.
Ibarra, Oscar H. and Kim, *Chul E. Fast Approximation Algorithms for the Knapsack and Sum of Subset Problems*, J. ACM, Volume 22 Issue 4, Oct. 1975.

4.  Vijay V. Vazirani. *Approximation Algorithms*. Georgia Institute of Technology, College of Computing.

5.  Eugene L. Lawler. *Fast Approximation Algorithms for Knapsack Problems*.